In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
tf.__version__

'2.17.0'

In [3]:
churn_df_original = pd.read_csv('Churn_Modelling.csv')
churn_df = churn_df_original.copy()
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
churn_df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
churn_df.duplicated().unique()

array([False])

In [7]:
X = churn_df.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y = churn_df['Exited']

In [8]:
unique = X.copy()
for col in X.columns:
    unique[col] = pd.Series(X[col].unique())
unique

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619.0,France,Female,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,608.0,Spain,Male,41.0,1.0,83807.86,3.0,0.0,0.0,112542.58
2,502.0,Germany,NaN,39.0,8.0,159660.80,2.0,NaN,NaN,113931.57
3,699.0,NaN,NaN,43.0,7.0,125510.82,4.0,NaN,NaN,93826.63
4,850.0,NaN,NaN,44.0,4.0,113755.78,NaN,NaN,NaN,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101699.77
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42085.58
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92888.52
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38190.78


In [9]:
X = pd.get_dummies(X,columns=['Gender'],drop_first=True)
X = pd.get_dummies(X,columns=['Geography'],drop_first=False)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)
X.shape

(10000, 12)

In [12]:
ann =  tf.keras.models.Sequential()
# input layer
ann.add(tf.keras.layers.Dense(units=12))
# hidden layer
ann.add(tf.keras.layers.Dense(units=10,activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [13]:
ann.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
ann.fit(scaled_X_train,y_train,batch_size=64,epochs=100)

Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8089 - loss: 0.4381  
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.8097 - loss: 0.4365
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8073 - loss: 0.4328
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8167 - loss: 0.4337
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.8156 - loss: 0.4275
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.8207 - loss: 0.4265
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8059 - loss: 0.4443
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.8128 - loss: 0.4295
Epoch 9/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.8176 - loss: 0.4178
Epoch 10/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.8191 - loss: 0.4260
Epoch 11/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.8101 - loss: 0.4334
Epoch 12/100
110/110 ━━━━